In [2]:
import random as rd
import numpy as np
import pandas as pd
import os

from scoring import Metrics
from classification import Random

In [53]:
a = np.array([0, 1, 2])
np.power(a, 2)

array([0, 1, 4])

In [65]:
# Kernel tests


data = np.array([
    [28, 3, 10, 0],
    [48, 1, 23, 1],
    [42, 10, 12, 1],
    [32, 11, 7, 1],
    [1, 7, 123, 0],
    [34, 2, 17, 1],
    [35, 3, 100, 0],
    [13, 12, 12, 0],
    [83, 2, 3, 1],
    [39, 12, 88, 1],
    [128, 23, 33, 0],
    [28, 3, 10, 0],
    [48, 1, 23, 1],
    [42, 10, 12, 1],
    [32, 11, 7, 1],
    [1, 7, 123, 0],
    [34, 2, 17, 1],
    [35, 3, 100, 0],
    [13, 12, 12, 0],
    [83, 2, 3, 1],
    [39, 12, 88, 1],
    [128, 23, 33, 0],
    [28, 3, 10, 0],
    [48, 1, 23, 1],
    [42, 10, 12, 1],
    [32, 11, 7, 1],
    [1, 7, 123, 0],
    [34, 2, 17, 1],
    [35, 3, 100, 0],
    [13, 12, 12, 0],
    [83, 2, 3, 1],
    [39, 12, 88, 1]
])


# Test data
X_train, y_train = data[:14, :-1], data[:14, -1]
X0_train = X_train[y_train == 0]
X1_train = X_train[y_train == 1]
X_test, y_test  = data[:, :-1], data[:, -1]

# Fake train and test array sizes
N_train = X_train.shape[0]
N0_train = X0_train.shape[0]
N1_train = X1_train.shape[0]
N_test  = X_test.shape[0]


#---------


# Tile X_train
X_train_tiled = np.tile (X_train, (N_test, 1, 1))  # useless
X0_train_tiled = np.tile (X0_train, (N_test, 1, 1))
X1_train_tiled = np.tile (X1_train, (N_test, 1, 1))
print(f'TRAIN: Tiled\n\t{X0_train.shape} => {X0_train_tiled.shape}\n\t{X1_train.shape} => {X1_train_tiled.shape}')

# Tile X_test
X_test_tiled = np.transpose( np.tile (X_test, (N_train, 1, 1)), axes=(1, 0, 2))  # useless
X_test_tiled_0 = np.transpose( np.tile (X_test, (N0_train, 1, 1)), axes=(1, 0, 2))  # Replicate as many times as there are train samples, and transpose (flip on its side)
X_test_tiled_1 = np.transpose( np.tile (X_test, (N1_train, 1, 1)), axes=(1, 0, 2))
print(f'TEST:  Tiled\n\t{X_test.shape} => {X_test_tiled_0.shape}\n\t{X_test.shape} => {X_test_tiled_1.shape}')

# Calculate element-wise distances (compress RGB information)
dist_all = np.linalg.norm(X_train_tiled - X_test_tiled, axis=2)
dist_0   = np.linalg.norm(X0_train_tiled - X_test_tiled_0, axis=2)
dist_1   = np.linalg.norm(X1_train_tiled - X_test_tiled_1, axis=2)
print(dist_all.shape)



# Apply kernel function weighing
# HERE

def KERNEL_FUNC (x: np.ndarray):
    return np.exp(-1 * np.power(x, 2))

def KERNEL_CAPPED (x: np.ndarray):  # 1/x function, capped in [-1, 1]
    ret = np.ones(x.shape[0])
    ret[x <= 1] = np.abs(1 / x)
    return ret


# Sum across training datapoints
sum_all = np.sum(dist_all, axis=1)
sum_0 = np.sum(dist_0, axis=1)
sum_1 = np.sum(dist_1, axis=1)

p0 = sum_0 / sum_all  # each one is a sum of the distances on all the train vectors of class 0; each one is a normalising factor for the corresponding x_test vector
p1 = sum_1 / sum_all

# Estimate labels (using Bayes' estimation rule)
y_pred = (p0 < p1).astype(int)
print(y_pred)




TRAIN: Tiled
	(6, 3) => (32, 6, 3)
	(8, 3) => (32, 8, 3)
TEST:  Tiled
	(32, 3) => (32, 6, 3)
	(32, 3) => (32, 8, 3)
(32, 14)
[0.58529204 0.66858454 0.67326412 0.61686828 0.37936988 0.6354196
 0.40392939 0.51877491 0.57485304 0.4349285  0.45874511 0.58529204
 0.66858454 0.67326412 0.61686828 0.37936988 0.6354196  0.40392939
 0.51877491 0.57485304 0.4349285  0.45874511 0.58529204 0.66858454
 0.67326412 0.61686828 0.37936988 0.6354196  0.40392939 0.51877491
 0.57485304 0.4349285 ] [0.41470796 0.33141546 0.32673588 0.38313172 0.62063012 0.3645804
 0.59607061 0.48122509 0.42514696 0.5650715  0.54125489 0.41470796
 0.33141546 0.32673588 0.38313172 0.62063012 0.3645804  0.59607061
 0.48122509 0.42514696 0.5650715  0.54125489 0.41470796 0.33141546
 0.32673588 0.38313172 0.62063012 0.3645804  0.59607061 0.48122509
 0.42514696 0.5650715 ]
[0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 1 0 0 1]


In [2]:
DIRPATH = 'src'

# Import data
import_path = os.path.join(DIRPATH, 'export.npy')

data = np.load (import_path)
#data[:, :-1] = data[:, :-1]/255.  # normalise values
vals_nb = data.shape[0]

In [3]:
# Create train and test sets (19.2s in low power, 11.5s)

"""
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)
"""

# def create_sets(data, train_frac=0.05):

#     # Create (train, test)
#     data_shuffled = data.copy()

#     np.random.seed(0)
#     np.random.shuffle(data_shuffled)

#     train_nb = int(vals_nb * train_frac)
#     # print(f'training on {train_nb} vals')

#     return data_shuffled[:train_nb], data_shuffled[train_nb:]

# data_train, data_test = create_sets(data, train_frac=0.3)

'\nfrom sklearn.model_selection import train_test_split\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)\n'

In [4]:
import cv2

# Test image
image_test_path = 'src/ima_1e8ccf23-d106-4227-908c-e4fbbb1da5f8.jpg'

image_test = image_test_path.split('/')[-1]
image_test_name, _ = os.path.splitext(image_test)
id_test = image_test_name[4:]

# Import and flatten test image
image_test_path = os.path.join('src/sky-images', image_test)
image_test_array = cv2.imread (image_test_path)
image_test_shape = image_test_array.shape
image_test_array_flat = image_test_array.reshape(-1, 1, 3)# / 255.
image_test_array_flat = image_test_array_flat.squeeze()

In [5]:
from classifiers.lda import LDA_bin as LDA

RUN_LDA = False
if RUN_LDA:
    X_train, y_train = data_train[:, :-1], data_train[:, -1]
    X_test,  y_test  = data_test [:, :-1], data_test [:, -1]

    # Train LDA
    lda = LDA()
    lda.fit (X_train, y_train)

    # Test LDA
    pred_lda = lda.predict(X_test)
    metrics_lda = Metrics(pred_lda, y_test)
    print(metrics_lda)

    # Test on test image
    image_test_pred = lda.predict(image_test_array_flat)
    image_test_pred = image_test_pred.reshape(image_test_shape[:2])
    cv2.imwrite(f'output/{image_test_name}-pred-lda.png', image_test_pred * 255)

"""
Eval time = 0.4s
=================================
TP  = 2273658
FP  = 225912
FN  = 328428
TN  = 4121792
P   = 2602086
N   = 4347704
=================================
TPR = 0.8737828034892006
TNR = 0.9480387809289685
FPR = 0.05196121907103152
FNR = 0.1262171965107994
=================================
Precision           = 0.9096196545805879
False omission rate = 0.05196121907103152
Error rate          = 0.07976356120112982
Accuracy            = 0.9202364387988702
=================================
F1-score            = 0.8913411645159924
MCC                 = 0.8287908705201819
Kappa score         = 0.8283733047108975
""";

In [6]:
### QDA
from classifiers.qda import QDA_bin as QDA

RUN_QDA = False
if RUN_QDA:
    X_train, y_train = data_train[:, :-1], data_train[:, -1]
    X_test,  y_test  = data_test [:, :-1], data_test [:, -1]

    # Train QDA
    qda = QDA()
    qda.fit (X_train, y_train)

    # Test QDA
    pred_qda = qda.predict(X_test)
    metrics_qda = Metrics(pred_qda, y_test)
    print(metrics_qda)

    # Test on test image
    image_test_pred = qda.predict(image_test_array_flat)
    image_test_pred = image_test_pred.reshape(image_test_shape[:2])
    cv2.imwrite(f'output/{image_test_name}-pred-qda.png', image_test_pred * 255)

"""
Eval time = 0.4s
=================================
TP  = 2441037
FP  = 58533
FN  = 658013
TN  = 3792207
P   = 3099050
N   = 3850740
=================================
TPR = 0.7876726738839321
TNR = 0.9847995450225151
FPR = 0.015200454977484847
FNR = 0.21232732611606783
=================================
Precision           = 0.9765827722368248
False omission rate = 0.015200454977484847
Error rate          = 0.10310325923517114
Accuracy            = 0.8968967407648288
=================================
F1-score            = 0.8720138176907881
MCC                 = 0.8001029410421224
Kappa score         = 0.7873374479219594
""";

In [7]:
# ### TODO: k-NN


# from classifiers.trees import KDTree, DecisionTree

# data = np.array([
#     [28, 3, 10, 0],
#     [48, 1, 23, 1],
#     [42, 10, 12, 1],
#     [32, 11, 7, 1],
#     [1, 7, 123, 0],
#     [34, 2, 17, 1],
#     [35, 3, 100, 0],
#     [13, 12, 12, 0],
#     [83, 2, 3, 1],
#     [39, 12, 88, 1],
#     [128, 23, 33, 0]
# ])


# tree = KDTree (data, dimension=3)
# tree.grow()
# print(tree)



# # dtree = DecisionTree(data)
# # dtree.grow()
# # print(dtree)

gr
7
gr
3
gr
4
gr
2
gr
1


ZeroDivisionError: division by zero